In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Drop irreleant columns

data = data.drop(['RowNumber','CustomerId','Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
label_encoder_gender=LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data[['Gender']])
data.head()

/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
#Onehot Encoder for Geography column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder = geo_encoder.toarray()

In [6]:
onehot_encoder_geo.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encoded_df = pd.DataFrame(geo_encoder, columns=onehot_encoder_geo.get_feature_names_out())
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [9]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
with open('gender_label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

In [11]:
with open('geo_onehot_encoder.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

In [12]:
## Divide data into dependent and independent features

X = data.drop('Exited', axis=1)
y = data['Exited']


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [26]:
## Build ANN Model
model1 = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),##HL1 Connected to input layer
    Dense(32, activation='relu'), ## HL2
    Dense(1, activation='sigmoid'), ## Output layer
]

)

In [27]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
## compile the model
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model1.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
##Setup the TensorBoard
import tensorflow.keras.callbacks

log_dir = "log/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callbacks = TensorBoard(log_dir = log_dir, histogram_freq=1)

In [22]:
## Setup EarlyStopping

early_stopping_callbacks = EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)

In [29]:
## Training the Model

history = model1.fit(
    X_train, y_train, validation_data = (X_test, y_test), epochs=100,
    callbacks=[tensorflow_callbacks, early_stopping_callbacks]
)

Epoch 1/100
 15/250 [>.............................] - ETA: 0s - loss: 0.5643 - accuracy: 0.7229  

2025-06-11 00:51:32.353364: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


250/250 [==============================] - 1s 5ms/step - loss: 0.6291 - accuracy: 0.7605 - val_loss: 0.9378 - val_accuracy: 0.7180
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 4.2488 - accuracy: 0.7078 - val_loss: 6.9615 - val_accuracy: 0.6525
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 8.3652 - accuracy: 0.7085 - val_loss: 12.7606 - val_accuracy: 0.7005
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 12.0865 - accuracy: 0.7156 - val_loss: 45.6860 - val_accuracy: 0.5520
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 16.3006 - accuracy: 0.7207 - val_loss: 6.5403 - val_accuracy: 0.8030
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 20.4655 - accuracy: 0.7211 - val_loss: 11.1976 - val_accuracy: 0.7185
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 27.4912 - accuracy: 0.7193 - val_loss: 17.1463 - val_accuracy: 0.7785
Epo

In [31]:
model.save('model1.h5')

/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
# Load TensorBoard Extension
%load_ext tensorboard



In [33]:
%tensorboard --logdir log/fit